In [15]:
import pandas as pd
import requests
import numpy as np

In [ ]:
def convert_garz_to_review(data_path):

In [19]:
data = pd.read_csv("C:/Users/erknud3/fpl-optimization/model/data/Prediction_Data/player_ev_totals_gw3.csv")

In [20]:
data

,fpl_id,Player,web_name,team_name,position,now_cost,tsb,finishing,xMins,weighted_npxG,...,29,30,31,32,33,34,35,36,37,38
0,2,Gabriel Jesus,G.Jesus,Arsenal,FWD,6.8,0.9,0.91,19.36,0.47,...,1.69,1.76,1.59,1.71,1.77,1.65,1.74,1.51,1.72,1.75
1,3,Gabriel Magalhães,Gabriel,Arsenal,DEF,6.0,12.6,0.99,90.00,0.12,...,4.41,4.95,4.25,4.64,5.27,4.89,4.82,3.25,4.34,5.05
2,4,Kai Havertz,Havertz,Arsenal,FWD,8.1,17.9,0.94,90.00,0.40,...,4.06,4.35,3.71,4.16,4.41,3.95,4.27,3.39,4.18,4.31
3,6,Jurriën Timber,J.Timber,Arsenal,DEF,5.5,1.0,0.96,60.93,0.06,...,3.03,3.38,2.97,3.18,3.61,3.39,3.30,2.27,2.96,3.47
4,9,Gabriel Martinelli,Martinelli,Arsenal,MID,6.9,2.2,1.05,57.28,0.29,...,3.56,3.84,3.29,3.67,3.94,3.55,3.77,2.92,3.63,3.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348,621,João Félix,João Félix,Chelsea,MID,6.5,1.1,1.01,27.08,0.37,...,1.34,1.76,1.59,2.08,1.65,1.76,1.62,1.60,1.84,1.61
349,622,Sander Berge,Berge,Fulham,MID,5.0,0.0,1.01,25.45,0.07,...,1.19,1.07,1.14,1.15,1.17,1.21,1.14,1.20,1.14,1.10
350,625,İlkay Gündoğan,Gündogan,Man City,MID,6.5,0.3,0.98,21.22,0.20,...,1.37,1.51,1.36,1.37,1.32,1.42,1.45,1.44,1.43,1.35
351,631,Jack Clarke,J.Clarke,Ipswich,MID,5.5,0.0,1.09,51.09,0.08,...,1.98,1.84,2.04,1.78,1.72,1.79,1.83,1.94,1.96,2.02


In [21]:
static_url = "https://fantasy.premierleague.com/api/bootstrap-static/"
r = requests.get(static_url).json()
teams = r["teams"]

In [22]:
new_names = {i: i.strip() for i in data.columns}
data.rename(columns=new_names, inplace=True)

In [23]:
data["now_cost"] = pd.to_numeric(data["now_cost"], errors="coerce")

In [24]:
pos_fix = {"GKP": "G", "DEF": "D", "MID": "M", "FWD": "F"}
data["position"] = data["position"].replace(pos_fix)

In [30]:
data.columns

Index(['fpl_id', 'Player', 'web_name', 'team_name', 'position', 'now_cost',
       'tsb', 'finishing', 'xMins', 'weighted_npxG', 'weighted_xAG', '1', '2',
       '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15',
       '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27',
       '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38'],
      dtype='object')

In [32]:
gws = []
for i in data.columns:
    if i.isdigit():  # Check if the column name is a digit
        # Convert to string if the column is not already a string
        if data[i].dtype == 'object':
            data[f"{i}_Pts"] = (
                data[i].str.strip().replace({"-": 0}).astype(float)
            )
        else:
            data[f"{i}_Pts"] = data[i].replace({"-": 0}).astype(float)

        # Handle the 'xMins' column similarly
        if data["xMins"].dtype == 'object':
            data[f"{i}_xMins"] = (
                data["xMins"]
                .str.strip()
                .replace({"-": 0})
                .astype(float)
                .replace({np.nan: 0})
            )
        else:
            data[f"{i}_xMins"] = (
                data["xMins"]
                .replace({"-": 0})
                .astype(float)
                .replace({np.nan: 0})
            )
        
        gws.append(i)  # Append to gws if the column name is a digit

In [40]:
data["Name"] = data["Player"]
data["Value"] = data["now_cost"]
data["Pos"] = data["position"]
data["review_id"] = data["fpl_id"]

In [41]:
df_final = data[
        ["review_id", "Name", "Pos", "Value"]
        + [f"{gw}_{tag}" for gw in gws for tag in ["Pts", "xMins"]]
    ].copy()

In [44]:
df_final.replace({"-": 0}, inplace=True)
elements_data = r["elements"]
player_ids = [i["id"] for i in elements_data]
player_names = {i["id"]: i["web_name"] for i in elements_data}
player_pos = {i["id"]: i["element_type"] for i in elements_data}
player_price = {i["id"]: i["now_cost"] / 10 for i in elements_data}
pos_no = {1: "G", 2: "D", 3: "M", 4: "F"}
values = []
existing_players = df_final["review_id"].to_list()
for i in player_ids:
    if i not in existing_players:
        entry = {
            "review_id": i,
            "Name": player_names[i],
            "Pos": pos_no[player_pos[i]],
            "Value": player_price[i],
            **{f"{gw}_{tag}": 0 for gw in gws for tag in ["Pts", "xMins"]},
        }
        values.append(entry)

In [46]:
team_data = teams
team_dict = {i["code"]: i["name"] for i in team_data}
player_teams = {i["id"]: team_dict[i["team_code"]] for i in elements_data}
# Add missing players
# df_final = pd.concat([df_final, pd.DataFrame(values, columns=df_final.columns)], ignore_index=True)
df_final["Team"] = df_final["review_id"].map(player_teams)

df_final["fpl_id"] = df_final["review_id"]

df_final["Name"] = df_final["review_id"].replace(player_names)

df_final.set_index("fpl_id", inplace=True)

In [48]:
df_final.to_csv(f"C:/Users/erknud3/fpl-optimization/model/data/garz.csv")